In [54]:
import sqlalchemy as alch
from sqlalchemy import create_engine
import pandas as pd
from getpass import getpass  # To get the password without showing the input

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#sns.set_theme(style="whitegrid")

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression #this is actually used for classification

In [55]:
# password = getpass("INSERT YOUR F*KING PASSWORD: ") # I don't ahve a password for SQL so removed this

In [56]:
dbName = "sakila" # using DB sakila
connection_string = f"mysql+pymysql://root:@localhost/{dbName}" #removed the password part as I don't have one for SQL
engine = create_engine(connection_string)

### rentals_may as a dataframe

In [57]:
rentals_may_df = pd.read_sql_query( #creating a dataframe called rentals_may_df and reading the data from MySQL 
"""
SELECT * FROM rentals_may;
""", engine
)
rentals_may_df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


### rentals_june as a dataframe

In [58]:
rentals_june_df = pd.read_sql_query( # same as above, but for June
"""
SELECT * FROM rentals_june;
""", engine
)
rentals_june_df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


#### Check the number of rentals for each customer for May

In [59]:
rentals_may_df = pd.read_sql_query( #used the same logic here, where I used the SQL query I created in workbench and put it in Python 
"""
SELECT customer_id, COUNT(rental_id) AS num_rentals_may
FROM rentals_may
GROUP BY customer_id
ORDER BY num_rentals_may ASC;

""", engine
)
rentals_may_df

,customer_id,num_rentals_may
0,459,1
1,333,1
2,456,1
3,88,1
4,532,1
...,...,...
515,596,6
516,161,6
517,506,7
518,109,7


#### Check the number of rentals for each customer for June

In [60]:
rentals_june_df = pd.read_sql_query(
"""
SELECT customer_id, COUNT(rental_id) AS num_rentals_june
FROM rentals_june
GROUP BY customer_id
ORDER BY num_rentals_june ASC;

""", engine
)
rentals_june_df

,customer_id,num_rentals_june
0,370,1
1,168,1
2,369,1
3,302,1
4,394,1
...,...,...
585,267,9
586,457,9
587,213,9
588,454,10


#### Write a function that checks if customer borrowed more or less films in the month of June as compared to May

In [67]:
#first merge the 2 tables
rentals_may_june_df = rentals_may_df.merge(rentals_june_df, on = "customer_id", how ="inner")
rentals_may_june_df.head()

,customer_id,num_rentals_may,num_rentals_june
0,459,1,7
1,333,1,4
2,456,1,7
3,88,1,3
4,532,1,2


In [64]:
rentals["june-may"] = "More rented in June"

In [70]:
rentals["june-may"][rentals["num_rentals_june"] < rentals["num_rentals_may"]] = "More rented in May"
rentals.sample(50)

KeyError: 'num_rentals_may'

In [71]:
rentals["june-may"][rentals["num_rentals_june"] == rentals["num_rentals_may"]] = "Same"
rentals.sample(20)

KeyError: 'num_rentals_may'

In [ ]:

#was playing around with this, but it needs more work clearly

# def most_rentals(rentals_may_june_df, num_rentals_may, num_rentals_june):

#     if "num_rentals_may" < "num_rentals_june":
#         print("The customer rented more films in June")
#     else:
#         print("The customer rented more films in May")
    
#     return 